In [2]:
import torch
import tiktoken
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [11]:
# Read the text file
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Tokenize the text (simple whitespace tokenization)
def tokenize_text(text):
    tokens = text.split()
    return tokens

# Build a vocabulary and encode the text
def build_vocab(tokens):
    vocab = {token: idx for idx, token in enumerate(set(tokens))}
    return vocab

def encode_text(tokens, vocab):
    encoded = [vocab[token] for token in tokens]
    return torch.tensor(encoded, dtype=torch.long)

# Decode the tokens back to text
def decode_tokens(encoded_tensor, vocab):
    reverse_vocab = {idx: token for token, idx in vocab.items()}
    decoded_text = " ".join([reverse_vocab[idx.item()] for idx in encoded_tensor])
    return decoded_text

In [23]:
file_path = 'training_text.txt'
text = read_text_file(file_path)

text = text.strip("\ufeff")
text = text.replace("\n", " ")

In [18]:
len(text)

print(text[:100])

In my younger and more vulnerable years my father gave me some advice
that I’ve been turning over in


In [19]:
char = sorted(list(set(text)))
vocab_size = len(char)
print(''.join(char))
print(vocab_size)


 !$()*,-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXY[]abcdefghijklmnopqrstuvwxyzçéêô —‘’“”…
87


In [20]:
enc = tiktoken.get_encoding("gpt2")
enc.n_vocab

50257

In [25]:
encode = enc.encode("hello")
print(encode)
enc.decode(encode)

encode1 = enc.encode(text[:100])
print(encode1)
enc.decode(encode1)

[31373]
[818, 616, 7099, 290, 517, 8826, 812, 616, 2988, 2921, 502, 617, 5608, 326, 314, 447, 247, 303, 587, 6225, 625, 287]


'In my younger and more vulnerable years my father gave me some advice that I’ve been turning over in'